### GreyScale Convertion

In [1]:
import cv2

def rgb_to_grey(img):
    grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    return grey

### Smoothing - Median Filtering

In [2]:
import numpy as np
from PIL import Image

# Filter with kerel size 3
def get_median(dt):
#     med_lst = []
#     for i in range(dt.shape[0]):
#         for j in range(dt.shape[1]):

#             if (i != 0 and i < dt.shape[0]-1) and (j != 0 and j< dt.shape[1]-1):

#                 ngh_lst = [ dt[i-1][j-1], dt[i][j-1], dt[i+1][j-1], 
#                            dt[i+1][j], dt[i+1][j+1], dt[i][j+1], 
#                            dt[i-1][j+1], dt[i-1][j] ]

#                 n = len(ngh_lst)
#                 ngh_lst.sort()

#                 if n % 2 == 0:
#                     median1 = ngh_lst[n//2]
#                     median2 = ngh_lst[n//2 - 1]
#                     median = (median1 + median2)/2
#                 else:
#                     median = ngh_lst[n//2]

#                 med_lst.append(median)

#             else:
#                 med_lst.append(dt[i][j])

#     array = np.reshape(med_lst, (dt.shape[0], dt.shape[1]))
#     array = np.array(array, dtype = np.uint8)

    # median filtering in cv2
    # kernel size is 5
    array = cv2.medianBlur(dt, 5)
    
    return array

### Smoothing - Mode Filtering

In [3]:
import numpy as np
from PIL import Image
import statistics 

def get_mode(dt):
    mod_lst = []
    for i in range(dt.shape[0]):
        for j in range(dt.shape[1]):

            if (i != 0 and i < dt.shape[0]-1) and (j != 0 and j< dt.shape[1]-1):

                ngh_lst = [ dt[i-1][j-1], dt[i][j-1], dt[i+1][j-1], 
                           dt[i+1][j], dt[i+1][j+1], dt[i][j+1], 
                           dt[i-1][j+1], dt[i-1][j] ]

                mode = statistics.mode(ngh_lst)
                mod_lst.append(mode)

            else:
                mod_lst.append(dt[i][j])

    array = np.reshape(mod_lst, (dt.shape[0], dt.shape[1]))
    array = np.array(array, dtype = np.uint8)
    return array

### Smoothing - Average Filtering

In [4]:
import numpy as np
from PIL import Image
import statistics 

def get_average(dt_in):
    
    filter_h, filter_w = 3, 3

    filter_coeff = [[1, 1, 1], 
                    [1, 1, 1], 
                    [1, 1, 1]]

    pad_h, pad_w = int((filter_h-1)/2), int((filter_w-1)/2)

    data_h = int(dt_in.shape[0]+ (2 * pad_h))
    data_w = int(dt_in.shape[1] + (2 * pad_w))

    data = np.zeros([data_h , data_w])

    data[pad_h:int(data_h - pad_h), pad_w:int(data_w - pad_w)] = dt_in

    out_lst = []
    for i in range(dt_in.shape[0]):
        for j in range(dt_in.shape[1]):
            out_lst.append(np.sum((data[i:i + filter_h, j:j + filter_w])*filter_coeff)/(9))

    array = np.reshape(out_lst, (dt_in.shape[0], dt_in.shape[1]))
    array = np.array(array, dtype = np.uint8)
    
    return array

### Smoothing - Weighted Average Filtering

In [5]:
import cv2
import numpy as np
from PIL import Image
import statistics 

def get_waverage(dt_in):
    
    filter_h, filter_w = 3, 3

    filter_coeff = [[1, 2, 1], 
                    [2, 4, 2], 
                    [1, 2, 1]]

    pad_h, pad_w = int((filter_h-1)/2), int((filter_w-1)/2)

    data_h = int(dt_in.shape[0]+ (2 * pad_h))
    data_w = int(dt_in.shape[1] + (2 * pad_w))

    data = np.zeros([data_h , data_w])

    data[pad_h:int(data_h - pad_h), pad_w:int(data_w - pad_w)] = dt_in

    out_lst = []
    for i in range(dt_in.shape[0]):
        for j in range(dt_in.shape[1]):
            out_lst.append(np.sum((data[i:i + filter_h, j:j + filter_w])*filter_coeff)/(16))

    array = np.reshape(out_lst, (dt_in.shape[0], dt_in.shape[1]))
    array = np.array(array, dtype = np.uint8)
    
    return array

### Gario UI

In [6]:
def choice(image1, smoot):
    
    print("Operation is ", smoot)
    
    image2 = image1
    
    stat_out = " "
    
    if smoot == "Median":
        
        med_img = get_median(rgb_to_grey(image1))
        image2 = Image.fromarray(med_img,'L')
        stat_out = "Median Filter"

    elif smoot == "Mode":
        
        mod_img = get_mode(rgb_to_grey(image1))
        image2 = Image.fromarray(mod_img,'L')
        stat_out = "Mode Filter"
        
    elif smoot == "Average":
        
        avg_img = get_average(rgb_to_grey(image1))
        image2 = Image.fromarray(avg_img,'L')
        stat_out = "Average Filter"
        
    elif smoot == "Weighted Avg.":
        
        wavg_img = get_waverage(rgb_to_grey(image1))
        image2 = Image.fromarray(wavg_img,'L')
        stat_out = "Weighted Avg. Filter"
        
  
    return image2, stat_out

In [7]:
import gradio as gr

img = gr.inputs.Image()

title = "Face Mask - Spatial Domain Analysis(Smoothing)"

image1 = gr.inputs.Image(shape=(299, 299), type="numpy")
smoot = gr.inputs.Radio(['Median', 'Mode', "Average", "Weighted Avg."], label="Smoothing")

stat_out = gr.outputs.Textbox(type="auto", label="Statistics")
image2 = gr.outputs.Image(type="numpy", label="Output")

stat_out = gr.outputs.Textbox(type="auto", label="Comment")

gr.Interface(choice, inputs = [image1, smoot],   
             outputs = [image2, stat_out], 
             title=title, 
             live=True).launch(share=False)

IMPORTANT: You are using gradio version 1.4.0, however version 1.5.0 is available, please upgrade.
--------
Running locally at: http://127.0.0.1:7869/
To get a public link for a hosted model, set Share=True
Interface loading below...


(<Flask 'gradio.networking'>, 'http://127.0.0.1:7869/', None)

Operation is  Median
Operation is  Median
Operation is  Median
Operation is  Median
Operation is  Median
Operation is  Median
Operation is  Median
Operation is  Median
Operation is  Median
Operation is  Median
Operation is  Median
Operation is  Median
Operation is  Median
Operation is  Median
Operation is  Median
Operation is  Median
Operation is  Median
Operation is  Median
Operation is  Median
Operation is  Median
Operation is  Median
Operation is  Median
Operation is  Median
Operation is  Median
Operation is  Median
Operation is  Median
Operation is  Median
Operation is  Median
Operation is  Mode
Operation is  Mode
Operation is  Average
Operation is  Average
Operation is  Weighted Avg.
Operation is  Weighted Avg.
Operation is  Weighted Avg.
Operation is  Weighted Avg.
Operation is  Weighted Avg.
Operation is  Weighted Avg.
Operation is  Weighted Avg.
Operation is  Weighted Avg.
Operation is  Weighted Avg.


Contact me : www.athulmathew.com